In [ ]:
"""
Topic: AWS CloudWatch Logging & Monitoring
==========================================
Demonstrates how to publish custom metrics, write logs,
and create CloudWatch alarms programmatically using boto3.
"""

import boto3
import logging
import time
from botocore.exceptions import ClientError

# -------------------------------------------------------------------
# Step 1 – Setup Logging
# -------------------------------------------------------------------
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")
logger = logging.getLogger("cloudwatch_demo")

# -------------------------------------------------------------------
# Step 2 – Initialize CloudWatch Client
# -------------------------------------------------------------------
cloudwatch = boto3.client("cloudwatch")
logs = boto3.client("logs")

# -------------------------------------------------------------------
# Step 3 – Publish a Custom Metric
# -------------------------------------------------------------------
def publish_custom_metric(metric_name, value, namespace="ETL/Monitoring"):
    """Publish custom metric for ETL pipeline tracking."""
    try:
        response = cloudwatch.put_metric_data(
            Namespace=namespace,
            MetricData=[
                {
                    "MetricName": metric_name,
                    "Value": value,
                    "Unit": "Count"
                }
            ]
        )
        logger.info(f"✅ Published metric '{metric_name}' = {value}")
        return response
    except ClientError as e:
        logger.error(f"❌ Failed to publish metric: {e}")

# -------------------------------------------------------------------
# Step 4 – Create CloudWatch Log Group & Stream
# -------------------------------------------------------------------
def create_log_group_and_stream(log_group, log_stream):
    """Create a CloudWatch log group and stream."""
    try:
        logs.create_log_group(logGroupName=log_group)
    except logs.exceptions.ResourceAlreadyExistsException:
        pass
    try:
        logs.create_log_stream(logGroupName=log_group, logStreamName=log_stream)
        logger.info(f"✅ Log stream '{log_stream}' ready in '{log_group}'.")
    except logs.exceptions.ResourceAlreadyExistsException:
        logger.info("ℹ️ Log stream already exists.")

# -------------------------------------------------------------------
# Step 5 – Write Custom Logs
# -------------------------------------------------------------------
def write_logs(log_group, log_stream, message):
    """Write logs manually to CloudWatch."""
    timestamp = int(round(time.time() * 1000))
    logs.put_log_events(
        logGroupName=log_group,
        logStreamName=log_stream,
        logEvents=[
            {
                "timestamp": timestamp,
                "message": message
            }
        ]
    )
    logger.info(f"📝 Logged message: {message}")

# -------------------------------------------------------------------
# Step 6 – Create CloudWatch Alarm
# -------------------------------------------------------------------
def create_alarm(metric_name, namespace, threshold, topic_arn):
    """Create an alarm that triggers if metric exceeds threshold."""
    try:
        response = cloudwatch.put_metric_alarm(
            AlarmName=f"{metric_name}_Alarm",
            MetricName=metric_name,
            Namespace=namespace,
            Statistic="Sum",
            Period=60,
            EvaluationPeriods=1,
            Threshold=threshold,
            ComparisonOperator="GreaterThanThreshold",
            AlarmActions=[topic_arn],
            AlarmDescription=f"Alert if {metric_name} > {threshold}"
        )
        logger.info(f"🚨 Alarm '{metric_name}_Alarm' created successfully.")
        return response
    except ClientError as e:
        logger.error(f"❌ Failed to create alarm: {e}")

# -------------------------------------------------------------------
# Step 7 – Main Function
# -------------------------------------------------------------------
def main():
    log_group = "/aws/etl/cloudwatch_demo"
    log_stream = "etl_run_2025_11_09"
    metric_name = "GlueJobFailures"
    namespace = "ETL/Monitoring"

    # Step 1: Create log group & stream
    create_log_group_and_stream(log_group, log_stream)

    # Step 2: Publish logs
    write_logs(log_group, log_stream, "ETL job started")
    write_logs(log_group, log_stream, "Processing data...")
    write_logs(log_group, log_stream, "ETL job completed successfully ✅")

    # Step 3: Publish metrics
    publish_custom_metric(metric_name, 0, namespace)
    publish_custom_metric("LambdaInvocations", 5, namespace)

    # Step 4: Create alarm (example SNS ARN placeholder)
    sns_topic_arn = "arn:aws:sns:us-east-1:123456789012:ETLAlerts"
    create_alarm(metric_name, namespace, threshold=0, topic_arn=sns_topic_arn)

    logger.info("🏁 CloudWatch demo completed successfully.")

if __name__ == "__main__":
    main()
